# Introduction

Unlike most other languages that handles errors the same way using exception, <span style="color:lightgreen">*Rust groups errors into two major categories: recoverable and unrecoverable errors.*</span>
- **Recoverable Error**: e.g. a file not found error, we most likely just want to report the problem to the user and retry the operation. Rust provides the type `Result<T, E>` to handle recoverable errors.
- **Unrecoverable Errors**: symptoms of bugs, like trying to access a location beyond the end of an array, and so we want to immediately stop the program (with the `panic!` macro).



# `panic!` for Unrecoverable Errors

There are two ways to cause a panic in practice: by taking an action that causes our code to panic (such as accessing an array past the end) or by explicitly calling the `panic!` macro. By default, these panics will print a failure message, unwind, clean up the stack, and quit, but we can put 
```rust
[profile.release]
panic = 'abort'
```
into `Cargo.toml` to not clean up the stack and let the OS does it later.

Using `panic!`:

In [4]:
fn main() {
    panic!("crash and burn");
}

main()

thread '<unnamed>' panicked at src/lib.rs:3:5:
crash and burn
stack backtrace:
   0: rust_begin_unwind
             at /rustc/a28077b28a02b92985b3a3faecf92813155f1ea1/library/std/src/panicking.rs:597:5
   1: core::panicking::panic_fmt
             at /rustc/a28077b28a02b92985b3a3faecf92813155f1ea1/library/core/src/panicking.rs:72:14
   2: ctx::main
   3: run_user_code_2
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


<span style="color:skyblue">*A **backtrace** is a list of all the functions that have been called to get to this point. We can run a project with `RUST_BACKTRACE=1 cargo run` or even `RUST_BACKTRACE=full cargo run` to get a verbal backtrace*</span>

# `Result<T, E>` for Recoverable Errors
Most errors aren’t serious enough to require the program to stop entirely. Sometimes, when a function fails, it’s for a reason that you can easily interpret and respond to, with this we have the `Result<T, E>` enum (included in prelude):
```rust
enum Result<T, E> {
    Ok(T),
    Err(E),
}
```
where 
- `T` represents the type of the value that will be returned in a success case within the `Ok` variant, 
- and `E` represents the type of the error that will be returned in a failure case within the `Err` variant. 

In [17]:
use std::fs::File;

fn main() {
    let greeting_file_result = File::open("hello.txt");
    println!("{:?}", greeting_file_result);

    let greeting_file = match greeting_file_result {
        Ok(file) => file,
        Err(error) => panic!("Problem opening the file: {:?}", error),
    };
    println!("{:?}", greeting_file);

}

main()

Ok(File { fd: 3, path: "/home/dk/Desktop/projects/programming/rust/rust-book/chap09-error-handling/hello.txt", read: true, write: false })
File { fd: 3, path: "/home/dk/Desktop/projects/programming/rust/rust-book/chap09-error-handling/hello.txt", read: true, write: false }


()

## `match`ing on Different Errors
Notice below that we use multiple `match` to handle different types of errors

In [19]:
use std::fs::File;
use std::io::ErrorKind;

fn main() {
    let greeting_file_result = File::open("hello2.txt");

    let greeting_file = match greeting_file_result {
        Ok(file) => file,
        Err(error) => match error.kind() {  // error is an io::Error struct with the kind() method that returns io::ErrorKind enum
            ErrorKind::NotFound => match File::create("hello2.txt") {  // ErrorKind::NotFound indicates if the file exists or not 
                                                                       // In that case, we create the file if it does not exist
                Ok(fc) => fc,  // as File::create could also fail, we need a second arm in the inner match expression
                Err(e) => panic!("Problem creating the file: {:?}", e),
            },
            other_error => {  // other reasons, e.g. no permission to open the file, then panic
                panic!("Problem opening the file: {:?}", other_error);
            }
        },
    };
}

main()

()

### Closure: Alternatives to Using `match` with `Result<T, E>`
Notice that the code above has a lot of `match`es. We can write the same code with closures, which are used with many of the methods defined on `Result<T, E>`. These methods can be more concise than using `match` when handling `Result<T, E>` values in your code.  
Below we use closure and the `unwrap_or_else` method

In [ ]:
use std::fs::File;
use std::io::ErrorKind;

fn main() {
    let greeting_file = File::open("hello2.txt").unwrap_or_else(|error| {
        if error.kind() == ErrorKind::NotFound {
            File::create("hello.txt").unwrap_or_else(|error| {
                panic!("Problem creating the file: {:?}", error);
            })
        } else {
            panic!("Problem opening the file: {:?}", error);
        }
    });
}

main()

## `unwrap` and `expect`: Shortcuts for Panic on Error

Using `match` can make the code very long and repeated. The `unwrap` method is a shortcut method implemented just like the `match` expression:  
<span style="color:skyblue">*If the `Result` value is the `Ok` variant, `unwrap` will return the value inside the `Ok`.  If the `Result` is the `Err` variant, `unwrap` will call the `panic!` macro for us*</span>

In [22]:
use std::fs::File;

fn main() {
    let greeting_file = File::open("hello.txt").unwrap();
    println!("{:?}", greeting_file);
}

main()

File { fd: 3, path: "/home/dk/Desktop/projects/programming/rust/rust-book/chap09-error-handling/hello.txt", read: true, write: false }


()

<span style="color:skyblue">*Similarly, the `expect` method lets us also choose the `panic!` error message.*</span>

In [25]:
use std::fs::File;

fn main() {
    let greeting_file = File::open("hello3.txt")
        .expect("hello3.txt should be included in this project");
}

main()

thread '<unnamed>' panicked at src/lib.rs:4:10:
hello3.txt should be included in this project: Os { code: 2, kind: NotFound, message: "No such file or directory" }
stack backtrace:
   0: rust_begin_unwind
             at /rustc/a28077b28a02b92985b3a3faecf92813155f1ea1/library/std/src/panicking.rs:597:5
   1: core::panicking::panic_fmt
             at /rustc/a28077b28a02b92985b3a3faecf92813155f1ea1/library/core/src/panicking.rs:72:14
   2: core::result::unwrap_failed
             at /rustc/a28077b28a02b92985b3a3faecf92813155f1ea1/library/core/src/result.rs:1652:5
   3: <unknown>
   4: <unknown>
   5: <unknown>
   6: evcxr::runtime::Runtime::run_loop
   7: evcxr::runtime::runtime_hook
   8: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


## Propagating Errors

<span style="color:skyblue">***Propagating Errors** = When a function’s implementation calls something that might fail, instead of handling the error within the function itself, you can **return the error to the calling code** so that it can decide what to do. This gives more control to the calling code, where there might be more information or logic that dictates how the error should be handled than what you have available in the context of your code.*</span>  

Example: `read_username_from_file` reads a username from a file. If the file doesn’t exist or can’t be read, this function will return those errors to the code that called the function

In [30]:
use std::fs::File;
use std::io::{self, Read};

fn read_username_from_file_v1() -> Result<String, io::Error> {  // returns a Result<T, E>
    let username_file_result = File::open("hello.txt");

    let mut username_file = match username_file_result {
        Ok(file) => file,
        Err(e) => return Err(e),  // return the error if failed here
    };

    let mut username = String::new();

    match username_file.read_to_string(&mut username) {
        Ok(_) => Ok(username),
        Err(e) => Err(e),
    } // no semi-colon here, so we return username_file
}

read_username_from_file_v1()

Ok("dk")

## `?` Operator: A Shortcut for Propagating Errors

We can rewrite the `read_username_from_file` above with much less code using the `?` operator

In [32]:
use std::fs::File;
use std::io::{self, Read};

fn read_username_from_file_v2() -> Result<String, io::Error> {
    let mut username_file = File::open("hello.txt")?;
    let mut username = String::new();
    username_file.read_to_string(&mut username)?;
    Ok(username)
}

read_username_from_file_v2()

Ok("dk")

<span style="color:lightgreen">*The `?` placed after a `Result` value is defined to work in almost the same way as the `match` expressions we defined to handle the `Result` values in the first `read_username_from_file_v1`. **If the value of the `Result` is an `Ok`, the value inside the `Ok` will get returned from this expression**, and the program will continue. **If the value is an `Err`, the `Err` will be returned from the whole function** as if we had used the return keyword so the error value gets propagated to the calling code.*</span>

Furthermore, the `?` operator calls the `from` function (defined in the `From` std trait) to convert the the error type received into the error type defined in the return type of the current function. For example, we could change the `read_username_from_file_v2` function in return a custom error type named `OurError` that we define. If we also define `impl From<io::Error>` for `OurError` to construct an instance of `OurError` from an `io::Error`, then the`?` operator calls in the body of `read_username_from_file` will call from and convert the error types without needing to add any more code to the function.

We can even shorten the function more:

In [36]:
use std::fs::File;
use std::io::{self, Read};

fn read_username_from_file_v3() -> Result<String, io::Error> {
    let mut username = String::new();
    File::open("hello.txt")?.read_to_string(&mut username)?;
    Ok(username)
}

read_username_from_file_v3()

Ok("dk")

Since opens a file, creates a new `String`, then reads the content of the file into the String is a common operation, we have it already implemented in `fs::read_to_string`:

In [37]:
use std::fs;
use std::io;

fn read_username_from_file() -> Result<String, io::Error> {
    fs::read_to_string("hello.txt")
}
read_username_from_file()

Ok("dk")

## Where The `?` Operator Can Be Used

<span style="color:orange">***The `?` operator can only be used in functions whose return type is compatible with the value the `?` is used on. This is because the `?` operator is defined to perform an early return of a value out of the function***</span>

Below code will fail since the `?` operator follows the `Result` value returned by `File::open`, but this `main` function has the return type of `()`, not `Result`

In [43]:
use std::fs::File;

fn main() {
    let greeting_file = File::open("hello.txt")?;
}

main()

Error: the `?` operator can only be used in a function that returns `Result` or `Option` (or another type that implements `FromResidual`)

The error says that <span style="color:orange">*the `?` operator can only be used in a function that returns `Result` or `Option` (or another type that implements `FromResidual`)*</span>

<span style="color:lightgreen">*The behavior of the `?` operator when called on an `Option<T>` is similar to its behavior when called on a `Result<T, E>`: if the value is `None`, the `None` will be returned early from the function at that point. If the value is `Some`, the value inside the `Some` is the resulting value of the expression and the function continues*</span>

In [47]:
fn last_char_of_first_line(text: &str) -> Option<char> {
    text.lines().next()?.chars().last()
}
last_char_of_first_line("123")

Some('3')

> **Note 📝**: 
>
> <span style="color:orange">*You can use the `?` operator on a `Result` in a function that returns `Result`, and you can use the `?` operator on an `Option` in a function that returns `Option`, but you can’t mix and match. The `?` operator won’t automatically convert a `Result` to an `Option` or vice versa; in those cases, you can use methods like the `ok` method on `Result` or the `ok_or` method on `Option` to do the conversion explicitly.*</span>

## `Result<(), E>` for `main`

Since `main` can only return `()`, it has the `Result<(), Box<dyn Error>>`, where the `Box<dyn Error>` type is a trait object and can be thought of as "any kind of error"

In [49]:
use std::error::Error;
use std::fs::File;

fn main() -> Result<(), Box<dyn Error>> {
    let greeting_file = File::open("hello.txt")?;

    Ok(())
}

main()

Ok(())

<span style="color:lightgreen">*Using `?` on a `Result` value in a `main` function with the error type `Box<dyn Error>` is allowed, because it allows any `Err` value to be returned early*</span>.  

<span style="color:lightgreen">***When a `main` function returns a `Result<(), E>`, the executable will exit with a value of `0` if main returns `Ok(())` and will exit with a nonzero value if main returns an `Err` value***</span>

> **Final Note** 📝  
> <span style="color:skyblue">*The `main` function may return any types that implement the `std::process::Termination` trait, which contains a function `report` that returns an `ExitCode`*</span>